In [ ]:
import requests
dataset_url = "https://data.gov.il/api/3/action/datastore_search?resource_id=2d834439-3f88-45ef-bf6e-9924c2f7e8f9&limit={records_count}"
request_info = requests.get(dataset_url.format(records_count=0))
dataset_info_json = request_info.json()
dataset_records_total = dataset_info_json["result"]["total"]
dataset_all_records_url = dataset_url.format(records_count=dataset_records_total)
request_dataset = requests.get(dataset_all_records_url)
dataset = request_dataset.json()


In [ ]:
import pandas as pd

In [ ]:
pd_df = pd.DataFrame.from_dict(dataset["result"]["records"])

In [ ]:
pd_df.dtypes

In [ ]:
jar_path = "/home/masha/projects/databases-collection/databases-starter-kit/gui-clients/jdbc/"
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
spark_conf = SparkConf().setAppName("myApp")\
                      .setMaster("spark://rockmashine:7077")\
                      .set("spark.sql.repl.eagerEval.enabled", True)\
                      .set("spark.jars", jar_path + "postgresql-42.7.1.jar")
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark

In [ ]:
spark.sparkContext.getConf().getAll()

In [ ]:
pd_df.info()

In [ ]:
pd_df

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType

In [ ]:
schema = StructType([
    StructField("_id", IntegerType(), True),
    StructField("שם תחנה", StringType(), True),
    StructField("תאריך ניטור", StringType(), True),
    StructField("שיטת מדידה", StringType(), True),
    StructField("סמל פרמטר", StringType(), True),
    StructField("תוצאה", StringType(), True),
    StructField("יחידת מדידה", StringType(), True),
])

In [ ]:
sdf = spark.createDataFrame(dataset["result"]["records"], schema)
#spark.createDataFrame(pd_df, schema)

In [ ]:
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql.functions import col

In [ ]:
sdf.select("_id", "שם תחנה", "תאריך ניטור", to_timestamp("תאריך ניטור", "yyyy-MM-dd'T'HH:mm:SS").alias('dt'),"שיטת מדידה", "סמל פרמטר", "תוצאה", col("תוצאה").cast("Float").alias('val'), "יחידת מדידה" )

In [ ]:
new_sdf = sdf.select("_id", col("שם תחנה").alias("station_name"),
                     to_timestamp("תאריך ניטור", "yyyy-MM-dd'T'HH:mm:SS").alias('ts'),
                     col("שיטת מדידה").alias("meter_method"),
                     col("סמל פרמטר").alias("meter_object_abbreviation"),
                     col("תוצאה").cast("Float").alias('val'),
                     col("יחידת מדידה").alias("uom") )

In [ ]:
new_sdf

In [ ]:
rdd = new_sdf.rdd

In [ ]:
rdd.take(5)

In [ ]:
import getpass
pg_options = {
    "url": "jdbc:postgresql://192.168.49.2:5432/mydb",
    "user": "postgres",
    "password": getpass.getpass(prompt="postgre password"),
    "dbtable": "jordan_water_quality",
    "driver": "org.postgresql.Driver",
}

new_sdf.write\
    .format("jdbc")\
    .mode("overwrite")\
    .options(**pg_options)\
    .save()
